# Feature Engineering and Modelling

---

1. Import packages
2. Load data
3. Modelling

---

## 1. Import packages

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

---
## 2. Load data

In [3]:
df = pd.read_csv('./test_df.csv')
df.drop(columns=["Unnamed: 0"], inplace=True)
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,var_6m_price_mid_peak,churn,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.000000,0.000000,...,4.423670e+01,1,0,0,1,0,0,0,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2009-08-21,2016-08-30,2009-08-21,2015-08-31,2.280920,0.000000,...,0.000000e+00,0,1,0,0,0,0,1,0,0
2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,2010-04-16,2016-04-16,2010-04-16,2015-04-17,1.689841,0.000000,...,0.000000e+00,0,0,0,1,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2010-03-30,2016-03-30,2010-03-30,2015-03-31,2.382089,0.000000,...,0.000000e+00,0,0,0,0,1,0,1,0,0
4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2010-01-13,2016-03-07,2010-01-13,2015-03-09,2.650065,2.721811,...,4.860000e-10,0,1,0,0,0,0,1,0,0


In [4]:
df_copy = df.copy()

In [5]:
df_copy = df_copy.drop(columns={'date_activ', 'date_end', 'date_modif_prod', 'date_renewal'})

In [6]:
df.columns

Index(['id', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ',
       'date_end', 'date_modif_prod', 'date_renewal', 'forecast_cons_12m',
       'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_price_off_peak',
       'var_6m

---

## 3. Modelling

We now have a dataset containing features that we have engineered and we are ready to start training a predictive model. I will use a `Random Forest` classifier to classify clients by churn.

In [7]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### Data sampling

I will start by splitting the dataset into training and test datasets. This gives us the ability to see how well our model is able to generalise to new data, which is critical.

I will use a 75-25% split between train and test respectively.

In [8]:
# Make a copy of our data
train_df = df_copy.copy()

# Separate target variable from independent variables
y = df_copy['churn']
X = df_copy.drop(columns=['id', 'churn'])
print(X.shape)
print(y.shape)

(14606, 44)
(14606,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print("Training dataset shape:")
print(X_train.shape)
print(y_train.shape)
print("Test dataset shape:")
print(X_test.shape)
print(y_test.shape)

Training dataset shape:
(10954, 44)
(10954,)
Test dataset shape:
(3652, 44)
(3652,)


## Model training & Evaluation

A Random Forest sits within the category of `ensemble` algorithms because internally the `Forest` refers to a collection of `Decision Trees` which are tree-based learning algorithms.

The reason why an `ensemble` algorithm is powerful is because of the laws of averaging, weak learners and the central limit theorem. With `ensemble` methods, instead of banking on 1 single trained model, we can train 1000's of decision trees, all using different splits of the data and learning different patterns. It would be like asking 1000 people to all learn how to code. You would end up with 1000 people with different answers, methods and styles! The weak learner notion applies here too, it has been found that if you train your learners not to overfit, but to learn weak patterns within the data and you have a lot of these weak learners, together they come together to form a highly predictive pool of knowledge! This is a real life application of many brains are better than 1.

Now instead of relying on 1 single decision tree for prediction, the random forest puts it to the overall views of the entire collection of decision trees. Some ensemble algorithms using a voting approach to decide which prediction is best, others using averaging. 

As we increase the number of learners, the idea is that the random forest's performance should converge to its best possible solution.

Some additional advantages of the random forest classifier include:

- Rule-based approach instead of a distance calculation and so features do not need to be scaled
- Ability to handle non-linear parameters better than linear based models

On the flip side, some disadvantages of the random forest classifier include:

- The computational power needed to train a random forest on a large dataset is high, since we need to build a whole ensemble of estimators.
- Training time can be longer due to the increased complexity and size of thee ensemble

In [10]:
# Model training
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

Now let's evaluate how well this trained model is able to predict the values of the test dataset.

In [11]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [12]:
# Generate predictions
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [13]:
# Model Evaluation
accuracy = metrics.accuracy_score(y_pred, y_test)
accuracy

0.9030668127053669

In [14]:
print(metrics.classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95      3636
           1       0.04      0.88      0.07        16

    accuracy                           0.90      3652
   macro avg       0.52      0.89      0.51      3652
weighted avg       1.00      0.90      0.95      3652



In [15]:
predictions = model.predict(X)

In [16]:
general_accuracy = metrics.accuracy_score(predictions, y)
accuracy

0.9030668127053669

The general accuracy is the same as the accuracy of the model on test data. The model is doing a great job in terms of generalization.

In [17]:
print(metrics.classification_report(predictions, y))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     13537
           1       0.75      1.00      0.86      1069

    accuracy                           0.98     14606
   macro avg       0.88      0.99      0.92     14606
weighted avg       0.98      0.98      0.98     14606



In order to evaluate our model in a more concrete way we will use built-in method 'feature importance' that shows how much each feature contributes to the prediction.

In [18]:
feature_importances = pd.DataFrame({
    'features': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=True).reset_index()

In [19]:
feature_importances['importance'].head(10)

0    0.002585
1    0.002944
2    0.003027
3    0.003273
4    0.003813
5    0.004893
6    0.005044
7    0.005308
8    0.005412
9    0.005818
Name: importance, dtype: float64

In [20]:
def feature_importances_plot(feature_importances):
    plt.figure(figsize=(15, 20))
    plt.title('Feature Importance')
    plt.barh(range(len(feature_importances)), feature_importances['importance'], color='cadetblue', align='center')
    plt.yticks(range(len(feature_importances)), feature_importances['features'])
    plt.xlabel('Importance')

    sns.set_style('whitegrid')
    plt.tight_layout()
    plt.show()

The most 'Important' features for the Random Forest Classifier, above a 0.04 threshold, are :
- cons_12m
- net_margin
- forecast_meter_rent_12m
- forecast_cons_12m 
- margin_net_pow_ele 
- margin_gross_pow_ele

We can also notice the power of feature engineering while many engineered features are ranked among the most important features above a 0.02 threshold, we are talking about :
- months_activ
- months_modif_prod
- off_peak_peak_var_mean_diff
- var_year_price_off_peak
- off_peak_mid_peak_var_mean_diff
- var_year_price_off_peak_var
- offpeak_diff_dec_january_energy

It seems like our EDA wasn't that bad since we studied the variation of price by contract activation year. The month of activation of the contract and the month of modification of products are some of the most important feartures for our predictions.

It is important to note that our price sensitivity features are not the main reason why some customers churn

In [21]:
df['churn_predictions'] = predictions
df[df['churn_predictions']==1]

,id,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,churn,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,churn_predictions
5,1aa498825382410b098937d65c4ec26d,3.919235,0.0,3.300813,2011-12-09,2016-12-09,2015-11-01,2015-12-10,2.901970,3.300813,...,1,0,0,0,0,1,0,0,1,1
6,7ab4bf4878d8f7661dfc20e9b8e18011,4.654157,0.0,0.000000,2011-12-02,2016-12-02,2011-12-02,2015-12-03,3.906889,0.000000,...,1,0,0,1,0,0,0,0,1,1
25,389bbbe70248fbeecdf9bb1bd0d1da04,3.554489,0.0,2.509203,2010-11-17,2016-11-17,2010-11-17,2015-11-18,2.580731,2.509203,...,1,0,0,1,0,0,0,0,1,1
30,44e826a55734d0ca5eeafcae0e991a75,2.511883,0.0,0.000000,2009-07-07,2016-07-07,2015-05-23,2015-07-09,0.969416,0.000000,...,1,0,0,1,0,0,0,1,0,1
48,b96aa3a8655203318c6b853dbdb0ceb7,4.945365,0.0,3.784974,2010-01-15,2016-02-14,2013-09-14,2015-02-16,3.957671,3.784974,...,1,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14581,0af837073931dd6219536236fc246820,4.330759,0.0,0.000000,2011-11-24,2016-11-24,2013-10-28,2015-11-25,2.608130,0.000000,...,1,0,0,0,0,1,1,0,0,1
14588,2610e546b6d546f724cb0397dca1a14e,3.723209,0.0,0.000000,2013-01-09,2017-01-09,2015-11-18,2016-01-10,2.906976,0.000000,...,1,0,0,0,0,1,1,0,0,1
14593,c525fcb0860e5705d37743f5b5cadbfe,3.592177,0.0,2.193125,2012-10-23,2016-11-12,2015-09-19,2015-11-13,2.516099,0.000000,...,1,0,0,1,0,0,0,0,1,1
14602,d0a6f71671571ed83b2645d23af6de00,3.858778,0.0,2.260071,2012-08-27,2016-08-27,2012-08-27,2015-08-28,2.801191,2.260071,...,1,0,0,1,0,0,0,0,1,1


In [22]:
futur_churn_rate = len(df[df['churn_predictions']==1])/len(df) *100
futur_churn_rate

7.318910036971108

In [23]:
proba_predictions = model.predict_proba(X_test)
probabilities = proba_predictions[:, 1]

In [24]:
X_test = X_test.reset_index()
X_test.drop(columns='index', inplace=True)

In [25]:
X_test['churn'] = proba_predictions.tolist()
X_test['churn_probability'] = probabilities.tolist()
X_test['churn_probability']
X_test.to_csv('out_of_sample_data_with_predictions.csv')

In [26]:
X_test[X_test['churn_probability']>0.5]

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,churn,churn_probability
158,4.730944,0.000000,0.000000,2.470851,0.000000,0.0,1.279211,0.143460,0.000000,46.305378,...,0,0,1,0,0,0,0,1,"[0.019, 0.981]",0.981000
355,4.687243,0.000000,0.000000,2.522913,0.000000,0.0,1.232234,0.140621,0.000000,44.311378,...,0,1,0,0,0,0,1,0,"[0.148, 0.852]",0.852000
394,4.247261,0.000000,3.224792,3.356334,3.224792,24.0,2.155245,0.145677,0.125383,41.271364,...,0,0,1,0,0,0,0,1,"[0.439, 0.561]",0.561000
407,6.161926,4.790764,4.676108,3.530306,0.000000,0.0,1.201397,0.162033,0.084138,44.311378,...,1,0,0,0,0,1,0,0,"[0.395, 0.605]",0.605000
587,3.766190,0.000000,3.003461,2.353108,2.403121,0.0,1.212454,0.146348,0.000000,46.305378,...,0,0,1,0,0,1,0,0,"[0.38166666666666665, 0.6183333333333333]",0.618333
917,5.705621,0.000000,4.715176,0.000000,0.000000,0.0,2.162773,0.120372,0.103487,40.606701,...,0,0,1,0,0,0,0,1,"[0.212, 0.788]",0.788000
1252,5.032901,0.000000,4.556182,3.320339,2.733197,0.0,2.227270,0.100167,0.091892,58.995952,...,0,0,1,0,0,0,0,1,"[0.228, 0.772]",0.772000
1383,4.974014,0.000000,3.956168,2.704090,2.471292,0.0,1.171726,0.144902,0.000000,44.311378,...,0,0,1,0,0,0,0,1,"[0.039, 0.961]",0.961000
1536,5.705621,0.000000,4.715176,0.000000,0.000000,0.0,2.166341,0.120372,0.103487,40.606701,...,0,0,1,0,0,0,0,1,"[0.249, 0.751]",0.751000
1652,3.391641,0.000000,0.000000,2.573127,0.000000,0.0,1.224533,0.146694,0.000000,44.311378,...,0,0,1,0,0,0,0,1,"[0.467, 0.533]",0.533000
